In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code, category_id, 
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_category_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code,
            category_id) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_category_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


dict_test = {'weekly': ['2010-07-10', '2020-06-20'],
             'monthly': ['2010-07-31', '2020-04-30'],
             'quarterly' : ['2010-09-30', '2020-03-31'],
             'yearly' : ['2010-12-31', '2019-12-31']
}


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_date_range(date_list, granularity):
    result = []
    start = datetime.datetime.strptime(date_list[0], '%Y-%m-%d')
    end = datetime.datetime.strptime(date_list[1], '%Y-%m-%d')
    if granularity == 'weekly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(weeks=1)
    elif granularity == 'monthly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=1)
    elif granularity == 'quarterly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=3)
    elif granularity == 'yearly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=12)
    # print result
    return result


def check_store_unified_db_completeness(date_list, graularity):
    for date in date_list:
        end = date
        if graularity == 'weekly':
            start = datetime.datetime.strptime(date, '%Y-%m-%d') - relativedelta(weeks=1) + relativedelta(days=1)
            start = datetime.datetime.strftime(start, '%Y-%m-%d')
        elif graularity == 'monthly':
            start = date[:7] + str('-01')
        elif graularity == 'quarterly':
            start = datetime.datetime.strptime(date, '%Y-%m-%d') - relativedelta(months=2)
            start = datetime.datetime.strftime(start, '%Y-%m') + str('-01')
        elif graularity == 'yearly':
            start = date[:4] + str('-01-01')
        print start, end
        unified_result = query(aa_dsn, daily_sql.format(start, end))
        db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
        # print date
        # print unified_result[0][0]
        # print db_result
        if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
            print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
        else:
            test_result.append(date)
            print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])




key = 'quarterly'
value = ['2010-09-30', '2020-03-31']
check_store_unified_db_completeness(get_date_range(value, key), key)
print test_result

